In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtester_shared_equity import *
from trade_data import fetch_ohlcvs
from passivbot_isolated_margin import load_settings
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import PrettyPrinter
from common_functions import sort_dict_keys

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
user = 'your_user_name'
all_settings = load_settings(user)
settings = all_settings['default']
settings['max_entry_acc_val_pct_per_hour'] = all_settings['global']['max_entry_acc_val_pct_per_hour']
settings

In [ ]:
n_days = 360 * 4
symbols = sorted(set([c + '/BTC' for c in ['AAA', 'BBB', 'CCC', 'DDD']]))
settings['symbols'] = symbols
dfs = {s: prep_df(fetch_ohlcvs(s, n_days, no_download=True), settings) for s in symbols}
settings['precisions'] = {s: calc_price_precision(dfs[s].low[::100]) for s in dfs}

In [ ]:
df = merge_dfs(dfs)
print('n_days', (df.index[-1] - df.index[0]) / DAY_TO_MILLIS)

In [ ]:
# modify settings
settings['max_entry_acc_val_pct_per_hour'] = 0.002
settings['entry_vol_modifier_exponent'] = 10
settings['min_exit_cost_multiplier'] = 10
settings['min_entry_delay_hours'] = 6
settings['min_markup_pct'] = 0.005
settings['max_markup_pct'] = 0.1
settings['n_days_to_min_markup'] = 18

In [ ]:
logs, trades = backtest(df, settings)
ldf = pd.DataFrame(logs).set_index('timestamp')
ldf

In [ ]:
hwm = ldf.equity.max()
hwmi = ldf.equity.idxmax()
n_days_hwm = (hwmi - ldf.index[0]) / DAY_TO_MILLIS
adg = hwm ** (1 / n_days_hwm)
print(f'average daily gain {adg:.6f} high water mark {hwm}')
print(f'average yearly gain {adg**(365):.6f}')

ldf.equity.plot()

In [ ]:
print('lowest margin level', ldf.margin_level.min())
ldf.margin_level.plot()

In [ ]:
s = symbols[0]
print(s)
tdf = pd.DataFrame(trades[s]).set_index('timestamp').sort_index()
buys = tdf[tdf.side == 'buy']
sels = tdf[tdf.side == 'sel']
lv = tdf.long_vwap.dropna()
sv = tdf.shrt_vwap.dropna()
le = buys[buys.type == 'entry'] # long entry
lx = sels[sels.type == 'exit']  # long exit
se = sels[sels.type == 'entry'] # shrt entry
sx = buys[buys.type == 'exit']  # shrt exit

In [ ]:
lv.plot(style='r-')
le.price.plot(style='b.--')
lx.price.plot(style='ro')

In [ ]:
sv.plot(style='b-')
se.price.plot(style='r.--')
sx.price.plot(style='bo')

In [ ]:
bvwap = buys.cost.sum() / buys.amount.sum()
svwap = sels.cost.sum() / sels.amount.sum()
levwap = le.cost.sum() / le.amount.sum()
lxvwap = lx.cost.sum() / lx.amount.sum()
sevwap = se.cost.sum() / se.amount.sum()
sxvwap = sx.cost.sum() / sx.amount.sum()
print('lxvwap / levwap', lxvwap / levwap)
print('sevwap / sxvwap', sevwap / sxvwap)
print('sevwap / levwap', sevwap / levwap)
print('svwap / bvwap', svwap / bvwap)